In [18]:
import warnings
warnings.filterwarnings('ignore')

In [19]:
import pandas as pd

test_y_df = pd.read_csv("./datasets/titanic_gender_submission.csv")
test_df = pd.read_csv("./datasets/titanic_test.csv")
train_df = pd.read_csv("./datasets/titanic_train.csv")

In [20]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [21]:
train  = train_df[['Pclass', 'Sex', 'Age', 'Survived']]
test = test_df[['Pclass', 'Sex', 'Age']]
test_y = test_y_df['Survived']

In [22]:
train.isnull().sum()

Pclass        0
Sex           0
Age         177
Survived      0
dtype: int64

In [23]:
train = train.dropna(axis=0)

In [24]:
train.isnull().sum()

Pclass      0
Sex         0
Age         0
Survived    0
dtype: int64

In [25]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 714 entries, 0 to 890
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    714 non-null    int64  
 1   Sex       714 non-null    object 
 2   Age       714 non-null    float64
 3   Survived  714 non-null    int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 27.9+ KB


In [26]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
data = encoder.fit_transform(train['Sex'])

In [ ]:
data

In [28]:
train['Sex'] = data

In [30]:
train.head()

,Pclass,Sex,Age,Survived
0,3,1,22.0,0
1,1,0,38.0,1
2,3,0,26.0,1
3,1,0,35.0,1
4,3,1,35.0,0


In [39]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(train[['Age']])
data = scaler.transform(train[['Age']])

train['Age'] = data

In [40]:
train.head()

,Pclass,Sex,Age,Survived
0,3,1,0.271174,0
1,1,0,0.472229,1
2,3,0,0.321438,1
3,1,0,0.434531,1
4,3,1,0.434531,0


In [44]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [90]:
dt = DecisionTreeClassifier(random_state=42)
rf = RandomForestClassifier(random_state=42)
lr = LogisticRegression()

X = train.drop('Survived', axis=1)
y = train['Survived']

result_dict = {
    'dt': [],
    'rf' : [],
    'lr' : []
}

for index, algorithm in enumerate([dt, rf, lr]):
    result = cross_val_score(algorithm, X, y)
    if index == 0:
        result_dict['dt'] = result
    elif index == 1:
        result_dict['rf'] = result
    else:
        result_dict['lr'] = result

In [ ]:
# result = {}
# for i, j in enumerate([dt, rf, lr]) :
#   result[i] = cross_val_score(j, X, y)
# result

In [59]:
import numpy as np
print((np.mean(result_dict['dt'])))
print((np.mean(result_dict['rf'])))
print((np.mean(result_dict['lr'])))

0.8081552250566336
0.8109425785482124
0.7843494533635379


In [60]:
test

,Pclass,Sex,Age
0,3,male,34.5
1,3,female,47.0
2,2,male,62.0
3,3,male,27.0
4,3,female,22.0
...,...,...,...
413,3,male,NaN
414,1,female,39.0
415,3,male,38.5
416,3,male,NaN


In [61]:
test['Survived'] = test_y_df['Survived']

In [63]:
test.dropna(axis=0, inplace = True)

In [69]:
test['Sex']

Pclass      0
Sex         0
Age         0
Survived    0
dtype: int64

In [71]:
# 성별 숫자로 변환
encoder = LabelEncoder()
data = encoder.fit_transform(test['Sex'])
test['Sex'] = data

In [73]:
# 나이 정규화
scaler = MinMaxScaler()
scaler.fit(test[['Age']])
data = scaler.transform(test[['Age']])

test['Age'] = data

In [82]:
X_test = test.drop('Survived', axis=1)
rf.fit(X,y)
pred_rf = rf.predict(X_test)

In [83]:
from sklearn.metrics import accuracy_score

accuracy_score(test['Survived'], pred_rf)

0.8403614457831325

In [84]:
# DecisionTree
dt.fit(X,y)
pred_dt = dt.predict(X_test)
accuracy_score(test['Survived'], pred_dt)

0.8192771084337349

In [86]:
# LogisticRegression
lr.fit(X,y)
pred_lr = lr.predict(X_test)
accuracy_score(test['Survived'], pred_lr)

0.927710843373494